### Learning ipyrad output files

**20170517**

I want to get the hang of navigating and plotting things from the output files in ipyrad.



In [4]:
cd /mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUN1_py

/mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUN1_py


First, I can get the summary stats and paths to all the output files using the ``-r`` flag:

In [5]:
!ipyrad -p params-cragig002.txt -r


Summary stats of Assembly cragig002
------------------------------------------------
         state  reads_raw  reads_passed_filter  clusters_total  \
FG001        6    4496623              4496623           28719   
FG009        6    5828500              5828500           31799   
FG100_B      6    4191123              4191123           27072   
FG102_B      6    3757306              3757306           32437   
FG205        6    4762949              4762949           28809   
FG212        6    4163068              4163068            1568   
Q314         6    3797205              3797205           32843   
Q324         6    4203350              4203350           47166   
Q330         6    3683576              3683576           63309   
Q339         6    6057345              6057345          142327   

         clusters_hidepth  hetero_est  error_est  reads_consens  
FG001               18878    0.021602   0.002395          16813  
FG009               19670    0.021421   0.002188       

Clustering is one of the most important and sensitive steps. So it's worth understanding what's in the s3_cluster_stats.txt files. I talked to Katherine on the phone to help make sense of the headers, because the documentation is not extremely thorough (the cost of the writers making everything sleek and simple!). So, here's the file:

In [6]:
!head ./cragig002_clust_0.82/s3_cluster_stats.txt

        clusters_total  hidepth_min clusters_hidepth avg_depth_total avg_depth_mj avg_depth_stat sd_depth_total sd_depth_mj sd_depth_stat filtered_bad_align
FG001            28719         10.0            18878          140.59       212.31         212.31        1057.95     1299.12       1299.12                  0
FG009            31799         10.0            19670          164.59       264.19         264.19        1525.65     1933.09       1933.09                  0
FG100_B          27072         10.0            18151          140.58       208.32         208.32         666.99      805.98        805.98                  0
FG102_B          32437         10.0            18407          103.80       180.36         180.36         877.05     1158.43       1158.43                  0
FG205            28809         10.0            18786          148.52       226.15         226.15        1278.66     1577.97       1577.97                  0
FG212             1568         10.0             1550      